# Dialogpt (training with every responses row will contain eight previous response as a context).

In [75]:
! pip install transformers==4.22.2
! pip install rouge-score
! pip install paddlepaddle
! pip install paddlenlp

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

## Reading and parsing data 

The parsing script is based on scipt from [this repository](https://github.com/Sanghoon94/DailyDialogue-Parser)

In [2]:
!git clone https://github.com/Sanghoon94/DailyDialogue-Parser.git
!curl http://yanran.li/files/ijcnlp_dailydialog.zip -o jcnlp_dailydialog.zip
!mkdir output
!mkdir output/train output/test output/validation 

!unzip /kaggle/working/jcnlp_dailydialog.zip
!unzip /kaggle/working/ijcnlp_dailydialog/test.zip -d /kaggle/working/ijcnlp_dailydialog/
!unzip /kaggle/working/ijcnlp_dailydialog/train.zip  -d /kaggle/working/ijcnlp_dailydialog/
!unzip /kaggle/working/ijcnlp_dailydialog/validation.zip  -d /kaggle/working/ijcnlp_dailydialog/

!python /kaggle/working/DailyDialogue-Parser/parser.py -i /kaggle/working/ijcnlp_dailydialog/train -o /kaggle/working/output/train
!python /kaggle/working/DailyDialogue-Parser/parser.py -i /kaggle/working/ijcnlp_dailydialog/test -o /kaggle/working/output/test
!python /kaggle/working/DailyDialogue-Parser/parser.py -i /kaggle/working/ijcnlp_dailydialog/validation -o /kaggle/working/output/validation

!gzip -d '/kaggle/working/output/train/dial.txt.gz' 
!gzip -d '/kaggle/working/output/validation/dial.txt.gz' 
!gzip -d '/kaggle/working/output/test/dial.txt.gz' 

Cloning into 'DailyDialogue-Parser'...
remote: Enumerating objects: 21, done.
remote: Total 21 (delta 0), reused 0 (delta 0), pack-reused 21
Receiving objects: 100% (21/21), done.
Resolving deltas: 100% (5/5), done.


## Delcare model inital configuration

Configuration and training scripts are mostly based on [script](https://github.com/huggingface/transformers/tree/main/examples/pytorch/language-modeling) and [transformer notebook](https://huggingface.co/transformers/v2.11.0/notebooks.html) from Huggingface.

In [6]:
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-small")

/opt/conda/lib/python3.7/site-packages/transformers/models/auto/modeling_auto.py:1252: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [8]:
import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

import re
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from paddlenlp.metrics import Distinct


from pathlib import Path

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

# Configs
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [9]:
class Args():
    def __init__(self):
        self.output_dir = '../tmp/output-small'
        self.model_type = 'gpt2'
        self.model_name_or_path = 'microsoft/DialoGPT-small'
        self.config_name = 'microsoft/DialoGPT-small'
        self.tokenizer_name = 'microsoft/DialoGPT-small'
        self.cache_dir = 'cached'
        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 4
        self.per_gpu_eval_batch_size = 4
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 3
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 3500
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False # Can set it as True if GPU is T4, which can increase the speed of training.
        self.fp16_opt_level = 'O1'

args = Args()

## Prepare Dataset

For the experiment, the following code will convert the dataset become every responce row will contain **eight** previous responces as a context since that the dataset have average eight turn conversation. 

In [10]:
def read_data(type):
  df = pd.read_csv(f'/kaggle/working/output/{type}/dial.txt', sep='/n', names=['diag'])
  df = df.drop_duplicates(subset=['diag'])
  df = df.reset_index(drop = True)
  return df

In [11]:
def data_preparing(df, n=8-1): # since the average approximate is 8 turns 
  context = []
  col = ['response', 'context']
  col = col + ['context/'+str(idx) for idx in range(n-1)]
  for idx in range(n, len(df['diag'])):
    temp = []
    for k in range(idx, idx-n-1, -1):
      temp.append(df['diag'][k])
    context.append(temp)
  result_df = pd.DataFrame.from_records(context, columns=col)
  return result_df

In [12]:
trn_df = data_preparing(read_data('train'))
val_df = data_preparing(read_data('validation'))

/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [13]:
trn_df.head()

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
0,"Sounds great to me ! If they are willing , we ...",That's a good idea . I hear Mary and Sally oft...,I suggest a walk over to the gym where we can ...,I guess you are right.But what shall we do ? I...,Do you really think so ? I don't . It will jus...,What do you mean ? It will help us to relax .,You know that is tempting but is really not go...,"Say , Jim , how about going for a few beers af..."
1,Good.Let ' s go now .,"Sounds great to me ! If they are willing , we ...",That's a good idea . I hear Mary and Sally oft...,I suggest a walk over to the gym where we can ...,I guess you are right.But what shall we do ? I...,Do you really think so ? I don't . It will jus...,What do you mean ? It will help us to relax .,You know that is tempting but is really not go...
2,All right .,Good.Let ' s go now .,"Sounds great to me ! If they are willing , we ...",That's a good idea . I hear Mary and Sally oft...,I suggest a walk over to the gym where we can ...,I guess you are right.But what shall we do ? I...,Do you really think so ? I don't . It will jus...,What do you mean ? It will help us to relax .
3,Can you do push-ups ?,All right .,Good.Let ' s go now .,"Sounds great to me ! If they are willing , we ...",That's a good idea . I hear Mary and Sally oft...,I suggest a walk over to the gym where we can ...,I guess you are right.But what shall we do ? I...,Do you really think so ? I don't . It will jus...
4,Of course I can . It's a piece of cake ! Belie...,Can you do push-ups ?,All right .,Good.Let ' s go now .,"Sounds great to me ! If they are willing , we ...",That's a good idea . I hear Mary and Sally oft...,I suggest a walk over to the gym where we can ...,I guess you are right.But what shall we do ? I...


In [14]:
val_df.head()

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
0,Good afternoon . This is Michelle Li speaking ...,1000 Yuan.Here you are .,How much would you like to change ?,RIB .,"Surely , of course . What kind of currency hav...","Well , that's too far.Can you change some mone...",There is one . 5 blocks away from here ?,"Good morning , sir . Is there a bank near here ?"
1,"This is Mr Meng speaking , Michelle .",Good afternoon . This is Michelle Li speaking ...,1000 Yuan.Here you are .,How much would you like to change ?,RIB .,"Surely , of course . What kind of currency hav...","Well , that's too far.Can you change some mone...",There is one . 5 blocks away from here ?
2,"Oh , hello ! Sorry about that . I'm just calli...","This is Mr Meng speaking , Michelle .",Good afternoon . This is Michelle Li speaking ...,1000 Yuan.Here you are .,How much would you like to change ?,RIB .,"Surely , of course . What kind of currency hav...","Well , that's too far.Can you change some mone..."
3,That was quick ! I wasn't expecting it until l...,"Oh , hello ! Sorry about that . I'm just calli...","This is Mr Meng speaking , Michelle .",Good afternoon . This is Michelle Li speaking ...,1000 Yuan.Here you are .,How much would you like to change ?,RIB .,"Surely , of course . What kind of currency hav..."
4,"Yes , our application procedures have speeded ...",That was quick ! I wasn't expecting it until l...,"Oh , hello ! Sorry about that . I'm just calli...","This is Mr Meng speaking , Michelle .",Good afternoon . This is Michelle Li speaking ...,1000 Yuan.Here you are .,How much would you like to change ?,RIB .


Now, we will add special token 'EOS' between responses to let the model know the end of each response in a string. 

In [15]:
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):

        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

        directory = args.cache_dir
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            for _, row in df.iterrows():
                conv = construct_conv(row, tokenizer)
                self.examples.append(conv)

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

In [16]:

def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
    return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)

## Training process

In [17]:
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  
    model.resize_token_embeddings(len(tokenizer))


    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )


    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):

            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]

            if args.n_gpu > 1:
                loss = loss.mean()  
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step



def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
  
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
   

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [18]:
def main(df_trn, df_val):
    args = Args()
    
    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device


    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    set_seed(args)

    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    model = AutoModelWithLMHead.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir=args.cache_dir,
    )
    model.to(args.device)
    
    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)

        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    if args.do_train:
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        model = AutoModelWithLMHead.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN) 
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelWithLMHead.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results

In [19]:
main(trn_df, val_df)

/opt/conda/lib/python3.7/site-packages/transformers/models/auto/modeling_auto.py:1252: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/18062 [00:00<?, ?it/s]

Iteration:   0%|          | 0/18062 [00:00<?, ?it/s]

Iteration:   0%|          | 0/18062 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1909 [00:00<?, ?it/s]

{'perplexity_': tensor(11.4356)}

## perplexity of the model is 11.4356

## Testing process

In [20]:
def preparing_test_data(df):
    prompt, completion = [], []
    for index, row in df.iterrows():
        if index % 2 == 0:
          prompt.append(row['diag']);
        else:
          completion.append(row['diag']);
    return prompt, completion

In [21]:
test_query, test_response = preparing_test_data(read_data('test'))

/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


Now, we will store the user input tokens to the chat history that to perform multi-turn conversation.

In [23]:
test_chatbot = []

for i in tqdm(range(len(test_response))):
  tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
  model = AutoModelWithLMHead.from_pretrained('../tmp/output-small')
  bot_input_ids = tokenizer.encode(test_query[i] + tokenizer.eos_token, return_tensors='pt')
  print("Question: {} \n".format(test_query[i]))
  print("Reference:  {} \n".format(test_response[i]))


  # generated a response while limiting the total chat history to 1000 tokens, 
  chat_history_ids = model.generate(
      bot_input_ids, max_length=100,
      pad_token_id=tokenizer.eos_token_id,  
      no_repeat_ngram_size=3,       
      do_sample=True, 
      top_k=10, 
      top_p=0.7,
      temperature = 0.9
  )

  # pretty print last ouput tokens from bot
  print("Predict: {} \n\n".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
  test_chatbot.append(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True))

  # reference = [test_response[i].split()]
                            
  # #candidatie = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True).split()

  # bleu_1.append(sentence_bleu(reference, tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True).split()
  #               , weights=(1, 0, 0, 0)))
  # bleu_2.append(sentence_bleu(reference, tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True).split()
  #               , weights=(0.5, 0.5, 0, 0)))  
  # bleu_4.append(sentence_bleu(reference, tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True).split()
  #               , weights=(0.25, 0.25, 0.25, 0.25)))
  
print(len(test_chatbot))

  0%|          | 0/3727 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/transformers/models/auto/modeling_auto.py:1252: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Patient: Hey man , you wanna buy some weed ? 

Reference:  Some what ? 

Predict: Yeah, I ’ m not sure what kind of strain to buy. 




A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Patient: Weed ! You know ? Pot , Ganja , Mary Jane some chronic ! 

Reference:  Oh , umm , no thanks . 

Predict: What do you mean? You haven't even had a cigarette for a week. 




...

Patient: yes , it was wonderful but it's already been a year . How time flies ! 

Reference:  why not go again to celebrate out one-year anniversary ? We can go to the same beach , stay in the same hotel and enjoy a dinner in the same restaurant . 

Predict: I know. I have been working in the company for nearly a year and a half and I still haven't received my salary. 




A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Patient: are you kidding ? Can you afford it ? Do you think we can get a room on such short notice ? 

Reference:  never mind that , I'll take care of it . Are you available next week ? 

Predict: sure. Look at this place. It's got all the modern conveniences, and it's got a big kitchen, and a big garden. 




A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Patient: yeah , I think so . 

Reference:  ok . I'll make the arrangements . It will be great . 

Predict: I'm glad you're getting better. 


3727


In [76]:
# confirm all the test case is finished
print(len(test_response))
print(len(test_chatbot))

3727
3727


## Evaluation

As the dataset contains some **invalid punctation** like sentences with extra whitespace, so that we will preprocessing the reference response before the evaluation.

In [25]:
def preprocessing(s):
    s = s.strip()  # Remove leading and trailing whitespace
    s = re.sub(r'[^\w\s]', '', s)  # Remove all punctuation marks
    s = re.sub(r'\s+', ' ', s)  # Replace multiple whitespace characters with a single space
    s = re.sub(r'(\w+)\.(\w+)', r'\1 . \2', s)
    s = re.sub(r'(\w+)-(\w+)', r'\1 \2', s)
    s = s.replace('。', '.')
    s = s.replace('...', ',')
    s = s.replace(' p . m . ', ' pm ')
    s = s.replace(' P . m . ', ' pm ')
    s = s.replace(' a . m . ', ' am ')
    
    return s

In [26]:
def bleu(reference, candidate):
    smooth = SmoothingFunction().method4
    cumu_bleu1, cumu_bleu2, cumu_bleu3, cumu_bleu4, ind_bleu1, ind_bleu2, ind_bleu3, ind_bleu4 = [], [], [], [], [], [], [], []
    for ref, can in zip(reference, candidate):
        cumu_bleu1.append(sentence_bleu([preprocessing(ref).split()], preprocessing(can), weights=(1, 0, 0, 0) ,smoothing_function=smooth))
        cumu_bleu2.append(sentence_bleu([preprocessing(ref).split()], preprocessing(can), weights=(0.5, 0.5, 0, 0) ,smoothing_function=smooth))
        cumu_bleu3.append(sentence_bleu([preprocessing(ref).split()], preprocessing(can), weights=(0.33, 0.33, 0.33, 0) ,smoothing_function=smooth))
        cumu_bleu4.append(sentence_bleu([preprocessing(ref).split()], preprocessing(can), weights=(0.25, 0.25, 0.25, 0.25) ,smoothing_function=smooth))
        
        ind_bleu1.append(sentence_bleu([preprocessing(ref).split()], preprocessing(can), weights=(1, 0, 0, 0) ,smoothing_function=smooth))
        ind_bleu2.append(sentence_bleu([preprocessing(ref).split()], preprocessing(can), weights=(0, 1, 0, 0) ,smoothing_function=smooth))
        ind_bleu3.append(sentence_bleu([preprocessing(ref).split()], preprocessing(can), weights=(0, 0, 1, 0) ,smoothing_function=smooth))
        ind_bleu4.append(sentence_bleu([preprocessing(ref).split()], preprocessing(can), weights=(0, 0, 0, 1) ,smoothing_function=smooth))
        
    df = pd.DataFrame(list(zip(cumu_bleu1, cumu_bleu2, cumu_bleu3, cumu_bleu4, ind_bleu1,ind_bleu2, ind_bleu3, ind_bleu4)),
               columns =['Cumulative 1-gram', 'Cumulative 2-gram','Cumulative 3-gram', 'Cumulative 4-gram', 'Individual 1-gram', 'Individual 2-gram', 'Individual 3-gram', 'Individual 4-gram'])
    return df

In [77]:
def rouge(reference, candidate):
    measure_list = ['precision', 'recall', 'fmeasure']
    rouge1 = pd.DataFrame(columns=measure_list)
    rouge2 = pd.DataFrame(columns=measure_list)
    rougeL = pd.DataFrame(columns=measure_list)
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    for ref, can in zip(reference, candidate):
        scores = scorer.score(ref, can)
        rouge1 = rouge1.append(pd.DataFrame( [[scores['rouge1'].precision, scores['rouge1'].recall, scores['rouge1'].fmeasure]],
               columns=rouge1.columns),
               ignore_index = True)
        rouge2 = rouge2.append(pd.DataFrame( [[scores['rouge2'].precision, scores['rouge2'].recall, scores['rouge2'].fmeasure]],
               columns=rouge2.columns),
               ignore_index = True)
        rougeL = rougeL.append(pd.DataFrame( [[scores['rougeL'].precision, scores['rougeL'].recall, scores['rougeL'].fmeasure]],
               columns=rougeL.columns),
               ignore_index = True)

    return rouge1, rouge2, rougeL

In [ ]:
def distinct(response_list):
    distinct_1 = []
    distinct_2 = []

    for n in range(1, 3):
        distinct = Distinct(n_size=n)
        #print(n)
        for sentence in response_list:
            distinct.add_inst(sentence.split())
            distinct_1.append(distinct.score()) if n == 1 else distinct_2.append(distinct.score())
    return np.mean(distinct_1), np.mean(distinct_2)

In [80]:
bleu_result = bleu(test_response, test_chatbot)
mean_df2 = bleu_result.mean()
print(mean_df2)

Cumulative 1-gram    0.015820
Cumulative 3-gram    0.065983
Cumulative 4-gram    0.070455
Individual 1-gram    0.015820
Individual 3-gram    0.119376
Individual 4-gram    0.092046
dtype: float64


In [82]:
rouge1, rouge2, rougeL = rouge(test_response, test_chatbot)
print("rouge1 score: {:.3f}\n".format(rouge1.mean().recall))
print("rouge2 score: {:.3f}\n".format(rouge2.mean().recall))
print("rougeL score: {:.3f}\n".format(rougeL.mean().recall))

print("rouge1 summary:\n", rouge1.mean())
print("rouge2 summary:\n", rouge2.mean())
print("rougeL summary:\n", rougeL.mean())

rouge1 score: 0.141

rouge2 score: 0.032

rougeL score: 0.131



In [72]:
distinct_1, distinct_2 = distinct(test_chatbot)
print("Average Distinct_1 score of the test-set is {:.3f} \nAverage Distinct_2 score of the test-set is {:.3f}".format(distinct_1, distinct_2))

Average Distinct_1 score of the test-set is 0.202 
Average Distinct_2 score of the test-set is 0.537
